# 

# Propuesta de análisis para Sonora y Nuevo León

## Vivienda
- Promedio nacional
- Promedios por estado especificando clima 
- Consumo total por categoría de:
    - aa y naa
      - por tipologias 
    - consumos promedios de la vivienda con aa y naa por tipologia 
    - 


## Edificios APF

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from slugify import slugify  # pip install python-slugify
import seaborn as sns
import matplotlib.patches as mpatches


In [5]:
f = "../data/002_intermediate/apf_long.parquet"
apf = pd.read_parquet(f)
del apf["ent_mun"]
# del apf["consumo"]
# del apf["area"]
# del apf["municipio"]
# del apf["especifico"]
son = apf[apf.entidad=='SONORA']
nl = apf[apf.entidad=='NUEVO LEÓN']
nl.columns

Index(['generico', 'especifico', 'municipio', 'entidad', 'area', 'aa',
       'emensual', 'emensualm2', 'bioclima'],
      dtype='object')

In [6]:
nl.generico.unique()

array(['Oficina Pública', 'Salud', 'Comercio', 'Seguridad',
       'Educación y Cultura', 'Comunicaciones y Transporte'], dtype=object)

In [8]:
nl.especifico.unique()

array(['Administración Pública',
       'Asistencia social(centros de integración, de protección, orfanatos, asilos)',
       'Almacenes y bodegas',
       'Policía(central, comandancias, garitas, estaciones, corralones, módulos)',
       'Clínicas y centros de salud(consultorios, laboratorios)',
       'Educación superior', 'Hospitales',
       'Asistencia Social(centros de integración, de protección, orfanatos, asilos)',
       'Educación elemental(jardín de niños)',
       'Transportes terrestres, estaciones y terminales',
       'Comunicaciones(agencias y centrales, estaciones de radio y TV, estudios)'],
      dtype=object)

In [ ]:
apf_naa = apf[apf.aa=='NO']
apf_naa

In [ ]:
apf_naa

In [ ]:
#| label: fig-mensual-apf-personam2-sin-aa
#| fig-cap: Consumo total mensual por metro cuadrado de edificios de la APF para los diferentes bioclimas y sin uso de aire acondicionado.

# 1) Orden de bioclimas y paleta
climas = sorted(apf['bioclima'].unique())
colors = sns.color_palette('Set2', n_colors=len(climas))
palette_dict = {clima: colors[i] for i, clima in enumerate(climas)}

# 2) Catplot con orientación horizontal y facetas en columnas
g = sns.catplot(
    data=apf,
    x='emensualm2',        # consumo en el eje X
    y='bioclima',          # bioclima en el eje Y
    hue='bioclima',
    palette=palette_dict,
    kind='box',
    height=4,
    sharex=False,
    col='generico',
    whis=[25, 75],
    showfliers=False,
    col_wrap=3,
    legend=False           # quitamos la leyenda por defecto
)



# 3) Ajustar márgenes para dejar espacio a la derecha
g.fig.subplots_adjust(top=0.85, right=0.75, hspace=0.3)

# … pasos 4 y 5 sin cambios …

# 6) Prepara los parches
handles = [
    mpatches.Patch(color=palette_dict[clima], label=clima)
    for clima in climas
]
g.set_titles(template="{col_name}")
g.set_axis_labels(x_var='kWh/(m2 mes)', y_var='Bioclima')


# # 7) Leyenda vertical al lado derecho
# g.fig.legend(
#     handles=handles,
#     title='Bioclima',
#     loc='center left',
#     bbox_to_anchor=(1.02, 0.5),  # ligeramente fuera del área de dibujo
#     frameon=False
# )

plt.show()



In [ ]:

# función para calcular estadísticas
def calc_stats(df):
    return (
        df
        .groupby('bioclima')['emensualm2']
        .agg(
            P25     = lambda x: x.quantile(0.25),
            Mediana = lambda x: x.quantile(0.50),
            Media   = 'mean',
            P75     = lambda x: x.quantile(0.75)
        )
        .round(2)
        .reset_index()
    )

# para cada uso
for uso, grupo in apf_naa.groupby('generico'):
    stats = calc_stats(grupo)
    tag  = f"tbl-{slugify(uso, lowercase=True)}"
    caption = f"Estadísticas de consumo mensual (kWh/m²·mes) por bioclima para uso **{uso}**"
    
    # print(f"\n**Tabla: {uso}**  \n")
    print(stats.to_markdown(index=False, tablefmt="github"))
    print(f"\n: {caption} {{{{#{tag}}}}}\n")
